In [16]:
import os
os.chdir('../')

In [17]:

import mlflow
import joblib
from src.utils.utlis import *
from src.constants import *
from sklearn.metrics import accuracy_score
import pandas as pd
from mlflow.models.signature import infer_signature

In [18]:

model=joblib.load('artifacts/model_trainer/model.pkl')

In [28]:
from dataclasses import dataclass
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

@dataclass(frozen=True)
class EvaluationConfig:
    root_dir: Path
    model_path:Path
    test_data:Path
    mlflow_uri: str

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def eval_config(self):
        config = self.config.model_evaluation
        create_directories([config.root_dir])
        eval_confg=EvaluationConfig(
            root_dir=config.root_dir,
            model_path="artifacts/model_trainer/model.pkl",
            test_data="artifacts/data_transformation/final_test.csv",
            mlflow_uri=os.getenv('MLFLOW_TRACKING_URI')
           
        )    
        return eval_confg

In [33]:
class ModelEvaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config
        self.scores={}

    def save_score(self):
        
        save_json(path=Path("artifacts/model_evaluation/metrics.json"), data=self.scores) 

    def evaluate_model(self):
            # train_data=pd.read_csv(self.config.training_data_path)
            test_data=pd.read_csv(self.config.test_data)
            X_test = test_data.iloc[:, :-1].values
            y_test = test_data.iloc[:, -1].values   

            model=joblib.load(self.config.model_path)
            y_pred=model.predict(X_test)

            acc = accuracy_score(y_test, y_pred)
            self.scores["Accuracy Score"] = acc
            return model

    def log_with_mlflow(self,model):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        with mlflow.start_run():
            mlflow.log_param("Model Path", self.config.model_path)
            mlflow.log_metric("Accuracy Score", self.scores["Accuracy Score"])

            mlflow.sklearn.log_model(model, "random_forest_model")

    def evaluation(self):
        model = self.evaluate_model()

        # Save scores locally
        self.save_score()

        # Log evaluation with MLflow
        self.log_with_mlflow(model)
            

In [34]:
try:
    config = ConfigurationManager()
    eval_config = config.eval_config()
    evaluation = ModelEvaluation(eval_config)
    evaluation.evaluation()

except Exception as e:
   raise e

2024/12/25 17:04:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run fun-fish-472 at: https://dagshub.com/AIAkashMukherjee/Titanic-Survival.mlflow/#/experiments/0/runs/229b9fbfa045473296dbf491a45101f2
🧪 View experiment at: https://dagshub.com/AIAkashMukherjee/Titanic-Survival.mlflow/#/experiments/0
